In [64]:
import pandas as pd
import math
import numpy as np
import itertools

In [65]:
#statsDF - initial stats dataframe
#gamesDF - initial games dataframe
#newGamesDF - formatted games dataframe with game date as index and rest of record in one row
#finalStats - formatted stats dataframe with player's name as the index
#mainDF: The dataframe where the main analysis will take place.

In [66]:
stats = '/home/acer/github/nba/stats.csv'
games = '/home/acer/github/nba/games.csv'

In [67]:
#Dump the stats dataframe into a csv and clean it up
cols = ['PLAYER', 'MIN', 'FGM','FGA', 'FG%', '3PM', '3PA','3P%','FTM','FTA','FT%','OREB','DREB','REB','AST','TOV','STL','BLK','PF','PTS','+/-']
statsDF = pd.read_csv(stats, header= None, names=cols, index_col=False, error_bad_lines=False, delim_whitespace=True)
statsDF[cols] = statsDF[cols].replace({'\$': '', ',': ''}, regex=True)
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace('.','')
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace('-','')
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace("'",'')
statsDF['MIN'] = statsDF['MIN'].str.replace("'",'')
statsDF['MIN'] = statsDF['MIN'].replace({'\$': '', '-': ''}, regex=True)
#Drop the first line for the team name
if statsDF.iloc[0][0] == 'GLOSSARY':
    statsDF.drop([0,1], inplace=True)
else:
    statsDF.drop(0, inplace=True)

In [68]:
#The nba.com site changed its formatting so this is to remove non-existent rows

new_format_mask = 0
for index, row in statsDF.iterrows():
    if new_format_mask == 1:
        statsDF.drop(index, inplace=True)
        if row.iloc[0] == 'Totals:' and math.isnan(row.iloc[1]) == True:
            new_format_mask = 0 
        else:
            continue
    else:
        if row.iloc[0] == 'PLAYER' and row.iloc[1] != "MIN":
            new_format_mask += 1
        else:
            continue

for index, row in statsDF.iterrows():
    if row.iloc[0][3:4] == ":":
        statsDF.drop(index, inplace=True)

    

In [69]:
statsDF = statsDF[statsDF.PLAYER != 'PLAYER']
statsDF = statsDF[statsDF.PLAYER != 'Totals:']
statsDF = statsDF[statsDF.PLAYER != 'INACTIVE']
statsDF = statsDF.reset_index(drop=True)

In [82]:
#This will delete the second time a team is named. 
for index, row in statsDF.iterrows():
    try: 
        if row.iloc[1].isalpha() and statsDF.iloc[index + 1][1].isalpha() or row.iloc[1] == "76ers":
            home_team_length = index/2
            statsDF.drop(index, inplace=True)
            statsDF = statsDF.reset_index(drop=True)
            break
    except:
        continue

In [83]:
#Setting up the index on the main stats dataframe
mainDFIndex = []
for index, row in statsDF.iterrows():
    if row.iloc[0] == 'Nene':
        mainDFIndex.append(str(row['PLAYER']) + " " + str(row['MIN']))
    elif row.iloc[0].isalpha() == True and row.iloc[1].isalpha() == True:
        mainDFIndex.append(str(row['PLAYER']) + " " + str(row['MIN']))
print(mainDFIndex)

['Jayson Tatum', 'Al Horford', 'Aron Baynes', 'Jaylen Brown', 'Kyrie Irving', 'Daniel Theis', 'Terry Rozier', 'Abdel Nader', 'Shane Larkin', 'Guerschon Yabusele', 'Jabari Bird', 'Semi Ojeleye', 'JJ Redick', 'Robert Covington', 'Joel Embiid', 'Jerryd Bayless', 'Ben Simmons', 'Amir Johnson', 'Dario Saric', 'Markelle Fultz', 'TJ McConnell', 'Timothe LuwawuCabarrot', 'Justin Anderson', 'Jahlil Okafor', 'Nik Stauskas']


In [84]:
#Setup dicts from the DF
statsDF_Dict = statsDF.T.to_dict().values()

In [85]:
dict_list = []
#Iterate through the statsDF and only return lines with stats or lines where players sat
for index, i in enumerate(statsDF_Dict):
    if index == 1 or index % 2 != 0:
        dict_list.append(i)
    

In [86]:
finalStats = pd.DataFrame(dict_list)
finalStats.index = mainDFIndex
finalStats.drop(['+/-'], axis=1, inplace=True)
finalStats = finalStats.rename(columns={'PLAYER':'MIN','MIN':'FGM', 'FGM':'FGA', 'FGA':'FG%', 'FG%':'3PM', '3PM':'3PA', '3PA':'3P%', '3P%':'FTM', 'FTM':'FTA', 'FTA':'FT%', 'FT%':'OREB', 'OREB':'DREB', 'DREB':'REB', 'REB':'AST', 'AST':'TOV', 'TOV':'BLK', 'BLK':'PF', 'PF':'PTS', 'PTS':'+/-'})
#This will replace the nan's with DNP
finalStats = finalStats.fillna('NA')

In [87]:
finalStats

,FTM,3P%,3PA,TOV,PF,REB,3PM,FG%,FGA,OREB,FT%,FTA,FGM,DREB,PTS,MIN,+/-,AST,STL,BLK
Jayson Tatum,9,0.0,2,3,3,8,0,37.5,8,2,100,9,3,6,15,32:45,-4,0,3,0
Al Horford,4,42.9,7,4,2,9,3,31.3,16,3,100,4,5,6,17,36:25,5,3,1,0
Aron Baynes,0,0.0,0,1,5,8,0,62.5,8,5,0.0,1,5,3,10,13:39,-3,2,0,0
Jaylen Brown,0,25.0,4,0,2,8,1,30.8,13,0,0.0,3,4,8,9,36:56,15,0,0,2
Kyrie Irving,6,25.0,4,5,4,6,1,41.2,17,2,85.7,7,7,4,21,35:48,2,4,0,2
Daniel Theis,0,0.0,1,1,2,3,0,0.0,1,1,0.0,0,0,2,0,10:39,0,1,1,0
Terry Rozier,0,33.3,6,2,2,7,2,54.5,11,1,0.0,3,6,6,14,30:29,6,2,0,0
Abdel Nader,0,0.0,1,0,1,2,0,0.0,2,0,0.0,0,0,2,0,6:58,-2,0,0,0
Shane Larkin,0,100,2,1,0,2,2,80.0,5,0,0.0,0,4,2,10,19:26,16,4,0,0
Guerschon Yabusele,0,100,1,2,2,1,1,50.0,2,0,0.0,0,1,1,3,3:18,4,0,0,0


In [88]:
#GAMES CELL: This prepares the games section into a dataframe
#Passing in csv file and getting rid of the unused rows
gamesDF = pd.read_csv(games, header= None, names=['a','b','c','d','e','f','g','h','i'], index_col=False, error_bad_lines=False, delim_whitespace=True)
gamesDF.columns = [col.replace(',', '') for col in gamesDF.columns]
#if you want to operate on multiple columns, put them in a list like so:
games_cols = ['a','b','c','d','e','f','g','h','i']
# pass them to df.replace(), specifying each char and it's replacement:
gamesDF[games_cols] = gamesDF[games_cols].replace({'\$': '', ',': ''}, regex=True)
# This will add the winner to column b of each 26th row
for index, row in gamesDF.iterrows():
    if index % 26 == 0 or index == 0:
        if int(gamesDF['a'].iloc[index+4]) > int(gamesDF['a'].iloc[index + 8]):
            gamesDF['b'].iloc[index] = gamesDF['a'].iloc[index + 1]
        else:
            gamesDF['b'].iloc[index] = gamesDF['a'].iloc[index + 5]
    else:
        pass

In [89]:
#This cell will arrange the gamesDF into the actual formatted games dataframe, newGamesDF
#Setting up the index on the main stats dataframe
dateIndex = []
month = ['OCT', 'NOV', 'DEC', 'JAN', 'FEB']
for index, row in gamesDF.iterrows():
    for i in month:
        if gamesDF.iloc[index][0] == i:
            dateIndex.append(gamesDF.iloc[9,0] + " " + gamesDF.iloc[9,1][1] + ", " + gamesDF.iloc[9,2])
#The next lines will pull the data from the cells in the gamesDF and into the newGamesDF dataframe
gamesData = []
games = 0
for index, row in gamesDF.iterrows():
    if index == 0 or index % 26 == 0:
        gamesData.append(gamesDF.iloc[index+11,0])
        gamesData.append(gamesDF.iloc[index+12,0])
        gamesData.append(gamesDF.iloc[index,1])
        gamesData.append(gamesDF.iloc[index+11,1])
        gamesData.append(gamesDF.iloc[index+11,2])
        gamesData.append(gamesDF.iloc[index+11,3])
        gamesData.append(gamesDF.iloc[index+11,4])
        gamesData.append(gamesDF.iloc[index+12,1])
        gamesData.append(gamesDF.iloc[index+12,2])
        gamesData.append(gamesDF.iloc[index+12,3])
        gamesData.append(gamesDF.iloc[index+12,4])
        gamesData.append(gamesDF.iloc[index+8,0])
        gamesData.append(gamesDF.iloc[index+4,0])
        gamesData.append(gamesDF.iloc[index+22,2])
        gamesData.append(gamesDF.iloc[index+22,4])
        gamesData.append(gamesDF.iloc[index+22,6])
        games += 1
        
gamesData = np.array(gamesData)
gamesData = gamesData.reshape(games,16)
#Create an empty dataframe, based on the games date as the index
newGamesDF = pd.DataFrame(data = gamesData, columns=['Home','Away','W/L','1st Qtr H','2nd Qtr H','3rd Qtr H','4th Qtr H','1st Qtr A','2nd Qtr A','3rd Qtr A','4th Qtr A','Total H','Total A', 'Ref1','Ref2','Ref3'])
newGamesDF

,Home,Away,W/L,1st Qtr H,2nd Qtr H,3rd Qtr H,4th Qtr H,1st Qtr A,2nd Qtr A,3rd Qtr A,4th Qtr A,Total H,Total A,Ref1,Ref2,Ref3
0,BOS,PHI,BOSTON,24,22,23,33,21,29,22,20,92,102,"Kogut""""","Fitzgerald""""",Holtkamp


In [90]:
for index,row in finalStats.iterrows():
    finalStats['1st Qtr H'] = newGamesDF['1st Qtr H'][0]
    finalStats['2nd Qtr H'] = newGamesDF['2nd Qtr H'][0]
    finalStats['3rd Qtr H'] = newGamesDF['3rd Qtr H'][0]
    finalStats['4th Qtr H'] = newGamesDF['4th Qtr H'][0]
    finalStats['1st Qtr A'] = newGamesDF['1st Qtr A'][0]
    finalStats['2nd Qtr A'] = newGamesDF['2nd Qtr A'][0]
    finalStats['3rd Qtr A'] = newGamesDF['3rd Qtr A'][0]
    finalStats['4th Qtr A'] = newGamesDF['4th Qtr A'][0]
    finalStats['Total H'] = newGamesDF['Total H'][0]
    finalStats['Total A'] = newGamesDF['Total A'][0]
    finalStats['Ref1'] = newGamesDF['Ref1'][0]
    finalStats['Ref2'] = newGamesDF['Ref2'][0]
    finalStats['Ref3'] = newGamesDF['Ref3'][0]
    finalStats['W/L'] = newGamesDF['W/L'][0]
    finalStats['Home'] = newGamesDF['Home'][0]
    finalStats['Away'] = newGamesDF['Away'][0]
    

In [91]:
#This function will calculate the projected fantasy points per game
def fantasy_points(row):
    try:
        score = (int(row['3PM']) * 3) + (int(row['FGM']) * 2) + (int(row['REB']) * 1) + (int(row['AST']) * 1.5) + (int(row['BLK']) * 3) + (int(row['STL']) * 3) + (int(row['TOV']) * -1)
        return score
    except:
        pass

finalStats['Fantasy Score'] = finalStats.apply(fantasy_points, axis=1)
finalStats = finalStats.fillna('NA')

In [92]:
finalStats

,FTM,3P%,3PA,TOV,PF,REB,3PM,FG%,FGA,OREB,...,4th Qtr A,Total H,Total A,Ref1,Ref2,Ref3,W/L,Home,Away,Fantasy Score
Jayson Tatum,9,0.0,2,3,3,8,0,37.5,8,2,...,20,92,102,"Kogut""""","Fitzgerald""""",Holtkamp,BOSTON,BOS,PHI,20
Al Horford,4,42.9,7,4,2,9,3,31.3,16,3,...,20,92,102,"Kogut""""","Fitzgerald""""",Holtkamp,BOSTON,BOS,PHI,31.5
Aron Baynes,0,0.0,0,1,5,8,0,62.5,8,5,...,20,92,102,"Kogut""""","Fitzgerald""""",Holtkamp,BOSTON,BOS,PHI,20
Jaylen Brown,0,25.0,4,0,2,8,1,30.8,13,0,...,20,92,102,"Kogut""""","Fitzgerald""""",Holtkamp,BOSTON,BOS,PHI,25
Kyrie Irving,6,25.0,4,5,4,6,1,41.2,17,2,...,20,92,102,"Kogut""""","Fitzgerald""""",Holtkamp,BOSTON,BOS,PHI,30
Daniel Theis,0,0.0,1,1,2,3,0,0.0,1,1,...,20,92,102,"Kogut""""","Fitzgerald""""",Holtkamp,BOSTON,BOS,PHI,6.5
Terry Rozier,0,33.3,6,2,2,7,2,54.5,11,1,...,20,92,102,"Kogut""""","Fitzgerald""""",Holtkamp,BOSTON,BOS,PHI,26
Abdel Nader,0,0.0,1,0,1,2,0,0.0,2,0,...,20,92,102,"Kogut""""","Fitzgerald""""",Holtkamp,BOSTON,BOS,PHI,2
Shane Larkin,0,100,2,1,0,2,2,80.0,5,0,...,20,92,102,"Kogut""""","Fitzgerald""""",Holtkamp,BOSTON,BOS,PHI,21
Guerschon Yabusele,0,100,1,2,2,1,1,50.0,2,0,...,20,92,102,"Kogut""""","Fitzgerald""""",Holtkamp,BOSTON,BOS,PHI,4
